In [188]:
import os
import shutil
import importlib 

from climatools.cliradlw import (setup, pipeline)
from climatools.lblnew import setup_bestfit
import climatools.lblnew.pipeline as pipeline_lblnew

importlib.reload(setup)
importlib.reload(pipeline)

<module 'climatools.cliradlw.pipeline' from '/chia_cluster/home/jackyu/climatools/climatools/cliradlw/pipeline.py'>

In [189]:
def param_lblnew():
    return {'atmpro': 'mls',
  'band': '1',
  'commitnumber': 'd5931a1',
  'conc': None,
  'dv': 0.001,
  'klin': 0,
  'molecule': 'h2o',
  'ng_adju': [0, 0, 0],
  'ng_refs': [3, 3, 6],
  'nv': 1000,
  'option_compute_btable': 0,
  'option_compute_ktable': 1,
  'option_wgt_flux': 1,
  'option_wgt_k': 1,
  'ref_pts': [(1, 250), (30, 250), (300, 250)],
  'tsfc': 294,
  'vmax': 340,
  'vmin': 20,
  'w_diffuse': [(1.66, 1.66, 1.66),
   (1.66, 1.66, 1.66),
   (1.66, 1.66, 1.5, 1.5, 1.5, 1.5)],
  'wgt': [(0.2, 0.2, 0.2),
   (0.2, 0.2, 0.2),
   (0.2, 0.2, 0.65, 0.65, 0.65, 0.65)]}



In [171]:
os.listdir(pipeline_lblnew.get_dir_case(param=param_lblnew(), 
                             setup=setup_bestfit))

['kg_nonlin.dat',
 'kg_lin.dat',
 'output_wcoolrg.dat',
 'output_wfluxg.dat',
 'output_flux.dat',
 'output_fluxg.dat',
 'lblnew.exe',
 'output_coolr.dat',
 'output_coolrg.dat',
 'lblcom.f',
 'abscom.dat',
 'lblnew.f',
 'dgdgs.dat',
 'pt_ktable.dat']

In [172]:
!ls /chia_cluster/home/jackyu/radiation/offline_radiation_notebooks/longwave/lblnew_20160916/study__g1_threshold/h2o/conc_None/band01_wn_20_340/nv_1000/dv_0.001/ng_12/g_ascending_k_descending/refPTs_P_1_T_250__P_30_T_250__P_300_T_250/ng_refs_3__3__6/ng_adju_0__0__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_mls/wgt_k_1/wgt_0.2_0.2_0.2__0.2_0.2_0.2__0.2_0.2_0.65_0.65_0.65_0.65/wgt_flux_1/w_diffuse_1.66_1.66_1.66__1.66_1.66_1.66__1.66_1.66_1.5_1.5_1.5_1.5/option_compute_ktable_1/option_compute_btable_0/crd_00cecd2

ls: cannot access /chia_cluster/home/jackyu/radiation/offline_radiation_notebooks/longwave/lblnew_20160916/study__g1_threshold/h2o/conc_None/band01_wn_20_340/nv_1000/dv_0.001/ng_12/g_ascending_k_descending/refPTs_P_1_T_250__P_30_T_250__P_300_T_250/ng_refs_3__3__6/ng_adju_0__0__0/getabsth_auto__auto__auto/absth_dlogN_uniform__dlogN_uniform__dlogN_uniform/klin_none/atmpro_mls/wgt_k_1/wgt_0.2_0.2_0.2__0.2_0.2_0.2__0.2_0.2_0.65_0.65_0.65_0.65/wgt_flux_1/w_diffuse_1.66_1.66_1.66__1.66_1.66_1.66__1.66_1.66_1.5_1.5_1.5_1.5/option_compute_ktable_1/option_compute_btable_0/crd_00cecd2: No such file or directory


In [193]:
param = {'commitnumber': 'c8dce0e',
         'molecule': {'h2o': 'atmpro'},
         'band': [1,],
         'atmpro': 'mls',
         'tsfc': 294}

dir_fortran_lblnew = pipeline_lblnew.get_dir_case(param=param_lblnew(), 
                                              setup=setup_bestfit)

try:
    shutil.rmtree(pipeline.get_fortran_dir(param=param, setup=setup))
except FileNotFoundError:
    pass

try:
    shutil.rmtree(pipeline.get_analysis_dir(param=param, setup=setup))
except FileNotFoundError:
    pass


fproc = pipeline.run_fortran(param=param, setup=setup)

aproc = pipeline.run_analysis(param=param, 
                              dir_fortran_lblnew=dir_fortran_lblnew,
                              setup=setup)

{'h2o': 'wa(i,k)', 'co2': 0, 'o3': 0, 'n2o': 0, 'ch4': 0}
h2o

            wa(i,k)=wa(1,k)
wa(1,k)
wa(i,k)
co2

      co2 = 400.e-6
400.e-6
0.0
o3

            oa(i,k)=0.0
0.0
0.0
n2o

      n2o=0.0
0.0
0.0
ch4

      ch4=0.0
0.0
0.0
band

      bands(1:nband) = (/ 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1 /)


(/ 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1 /)
(/1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0/)
atmpro
atmosphere_profiles/mls75.pro
mls
mls
tsfc


      tb(1)=294.0
      

294.0
294
{'atmpro': 'mls',
 'band': [1],
 'commitnumber': 'c8dce0e',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}
/chia_cluster/home/jackyu/climatools/climatools/cliradlw/results_cliradlw.ipynb
/chia_cluster/home/jackyu/radiation/clirad-lw/LW/examples/lblnew/h2o_mls/band_1/atmpro_mls/cliradlw_c8dce0e
{'atmpro': 'mls',
 'band': [1],
 'commitnumber': 'c8dce0e',
 'molecule': {'h2o': 'atmpro'},
 'tsfc': 294}


In [194]:
out, err = fproc.communicate()
print(out.decode('utf-8'))
print(err.decode('utf-8'))

 before assigning
  -34.07336    
  0.0000000E+00



AttributeError: 'NoneType' object has no attribute 'decode'

In [195]:
out, err = aproc.communicate()
print(out.decode('utf-8'))
print()
print(err.decode('utf-8'))



[NbConvertApp] Converting notebook results_cliradlw.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 89877 bytes to results_cliradlw.ipynb

